# Counterexample X

### Installations

The below will install the necessary packages to run the notebook.

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install scipy

The following block imports the relevant modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.integrate import cumulative_trapezoid
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

The below helper functions will be useful later.

In [ ]:
def plot_surface_3d(F_vals, x_vals, y_vals=None, downsample=1, title='Surface Plot', z_title='Z', sphere_radius=None):
    """
    Plots a 3D surface for F_vals over the given x and y values.

    Parameters:
    - F_vals: 2D array of values representing F(x, y)
    - x_vals: 1D array for x-axis (or 2d precomputed grid coordinates)
    - y_vals: 1D array for y-axis (or 2d precomputed grid coordinates) (optional; defaults to x_vals)
    - downsample: int factor to reduce resolution (e.g., 2 means every 2nd point)
    - sphere_radius: float, optional — if provided, a translucent sphere of this radius is added
    """

    if y_vals is None:
        y_vals = x_vals

    if y_vals.ndim != x_vals.ndim:
        raise ValueError('x and y must have same number of dimensions.')

    # Downsample data if needed
    if downsample > 1:
        F_vals = F_vals[::downsample, ::downsample]

        if x_vals.ndim == 1:
            x_vals = x_vals[::downsample]
            y_vals = y_vals[::downsample]
        else:
            x_vals = x_vals[::downsample, ::downsample]
            y_vals = y_vals[::downsample, ::downsample]

    # Create 2D grid (X, Y)
    if x_vals.ndim == 1:
        X, Y = np.meshgrid(x_vals, y_vals, indexing='xy')
    else:
        X = x_vals
        Y = y_vals

    # Create surface plot list
    surfaces = [
        go.Surface(
            x=X,
            y=Y,
            z=F_vals,
            colorscale='Viridis',
            name='Main Surface'
        )
    ]

    # Optionally add a translucent sphere
    if sphere_radius is not None:
        u = np.linspace(0, np.pi, 50)
        v = np.linspace(0, 2 * np.pi, 50)
        U, V = np.meshgrid(u, v)
        Xs = sphere_radius * np.sin(U) * np.cos(V)
        Ys = sphere_radius * np.sin(U) * np.sin(V)
        Zs = sphere_radius * np.cos(U)

        surfaces.append(
            go.Surface(
                x=Xs,
                y=Ys,
                z=Zs,
                opacity=0.3,
                showscale=False,
                colorscale='Blues',
                name=f'Sphere r={sphere_radius}'
            )
        )

    # Create figure
    fig = go.Figure(data=surfaces)

    # Layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='x',
            yaxis_title='y',
            zaxis_title=z_title
        ),
        autosize=True,
        width=800,
        height=700
    )

    fig.show()


### Introduction

In this notebook, we shall construct a $C^1$ function $f:\mathbb{R}^3\rightarrow \mathbb{R}$ with the following properties:

 - $f(0,0,0)=0$.
 - The only critical point of $f$ occurs at the origin.
 - for every $\epsilon > 0$, the zero level set $f^{-1}(0)$ intersects the boundary of the ball $\partial B_\epsilon \subset \mathbb{R}^3$ tangentially.

To achieve this goal, we will build the function in three steps:

- **Step 1:** Construct a $C^1$ function $H : [0,1]^2 \rightarrow [0,1]$ with a critical point at every level $z \in [0,1]$. That is, for each $z \in [0,1]$, there exists $(x, y) \in [0,1]^2$ such that $H(x, y) = z$ and $\nabla H(x, y) = [0, 0]^\top$.

- **Step 2:** Construct a function $\tilde{f} : [0,1]^3 \rightarrow [0,1]$, with certain useful properties, such that $\tilde{f}^{-1}(0)$ contains the graph of $H$.

- **Step 3:** Transform the problem to spherical coordinates to obtain $f$ as the composition of $\tilde{f}$ with an appropriate coordinate transformation $T$.

We construct $f$ in a completely explicit and constructive manner, with all equations given in closed form.

### Step 1: Constructing H

To construct a function $H:\mathbb{R}^2\rightarrow \mathbb{R}$ with critical points at every level $z\in[0,1]$ we shall follow the construction suggested in [Grinberg, E.L. (2018)](https://www.tandfonline.com/doi/abs/10.1080/00029890.1985.11971725), but provide explicit equations throughout.

To begin, we shall first give definitions and then provide discussion to give some intuition behind the math. We start by letting $n\in\mathbb{N}_{>0}$ and $k \in \{0, 1, \dots, 2^n - 1\}$, and write the binary expansion of $k$ as:

\begin{equation}\nonumber
k = \sum_{j=1}^n r_j \cdot 2^{n-j}, \quad \text{where } r_j \in \{0,1\}.
\end{equation}
We then define:
\begin{equation}\nonumber
a_{n,k} := \sum_{j=1}^n r_j \cdot \frac{3}{5} \cdot \left( \frac{2}{5} \right)^{j-1},\quad\text{and}\quad b_{n,k} := a_{n,k} + \left( \frac{2}{5} \right)^n.
\end{equation}


Next, we define the following 'spike' function for $x \in [0,1]$, $n \in \mathbb{N}_{>0}$, and $k \in \{0, \dots, 2^{n-1} - 1\}$:

$$
\text{Spike}_{k,n}(x) = 
\begin{cases}
    0 & \text{if } x \leq \frac{1}{2}(a_{n,k} + b_{n,k} - w_n), \\
    2\frac{h_n}{w_n} \left(x - \frac{1}{2}(a_{n,k} + b_{n,k} - w_n) \right) & \text{if } \frac{1}{2}(a_{n,k} + b_{n,k} - w_n) \leq x \leq \frac{1}{2}(a_{n,k} + b_{n,k}), \\
    h_n - 2\frac{h_n}{w_n} \left(x - \frac{1}{2}(a_{n,k} + b_{n,k}) \right) & \text{if } \frac{1}{2}(a_{n,k} + b_{n,k}) \leq x \leq \frac{1}{2}(a_{n,k} + b_{n,k} + w_n), \\
    0 & \text{if } x \geq \frac{1}{2}(a_{n,k} + b_{n,k} + w_n).
\end{cases}
$$

where the height and width are given by:

$$
h_n := 4 \left( \frac{5}{6} \right)^{n+1}, \qquad w_n := \frac{2}{h_n \cdot 3^{n+1}}.
$$

To give some intuition as to what this looks like, we provide a function below:

In [ ]:
import numpy as np

def spike(x, n, k):
    """
    Vectorized version of the spike function Spike_{k,n}(x) for NumPy arrays.
    
    Parameters:
        x : np.ndarray
            Input values in [0, 1]
        n : int
            Scale level (must be >= 1)
        k : int
            Index in {0, ..., 2^n - 1}
            
    Returns:
        np.ndarray : Values of the spike function at x
    """
    x = np.asarray(x)
    
    # Convert k to binary vector r = [r1, ..., rn]
    r = [(k >> (n - j - 1)) & 1 for j in range(n)]
    
    # Compute a_{n,k}
    a_nk = sum(r_j * (3/5) * (2/5)**j for j, r_j in enumerate(r))
    
    # Compute b_{n,k}
    b_nk = a_nk + (2/5)**n
    
    # Compute height and width
    h_n = 4 * (5/6)**(n + 1)
    w_n = 2 / (h_n * 3**(n + 1))
    
    # Midpoint and boundaries
    mid = 0.5 * (a_nk + b_nk)
    left = mid - 0.5 * w_n
    right = mid + 0.5 * w_n

    # Initialize result
    y = np.zeros_like(x)

    # Linear rising edge
    mask_rise = (x > left) & (x <= mid)
    y[mask_rise] = 2 * h_n / w_n * (x[mask_rise] - left)

    # Linear falling edge
    mask_fall = (x > mid) & (x < right)
    y[mask_fall] = h_n - 2 * h_n / w_n * (x[mask_fall] - mid)

    # Peak (only applies if a value matches mid exactly, which is rare)
    y[np.isclose(x, mid)] = h_n

    return y

And an example plot:

In [ ]:
import matplotlib.pyplot as plt

n = 0
k = 0
x_vals = np.linspace(0, 1, 1000)
y_vals = spike(x_vals, n, k)

plt.plot(x_vals, y_vals)
plt.title(f"Spike function for n={n}, k={k}")
plt.xlabel("x")
plt.ylabel(f"Spike_{{{k},{n}}}(x)")
plt.grid(True)
plt.show()

The 'spike' function creates a triangular signal, with carefully chosen width and position, which encloses an area of exactly $(\frac{2}{3})^{n+1}$.

The width and position of the spikes have been carefully chosen so that we may define a function $h$ as follows:

\begin{equation}\nonumber 
    h(x)= \sum_{n=0}^{\infty}\sum_{i=0}^{2^n-1} \text{Spike}_{k,n}(x).
\end{equation}

In [ ]:
def compute_h(x, N_max=5):
    """
    Compute an approximation of h(x) = sum_{n=0}^{∞} sum_{k=0}^{2^n - 1} Spike_{k,n}(x)
    using a finite sum up to N_max.
    
    Parameters:
        x : np.ndarray
            Array of input values in [0,1].
        N_max : int
            Maximum value of n to include in the approximation (n=0 to N_max).
    
    Returns:
        np.ndarray : Approximated h(x) values.
    """
    x = np.asarray(x)
    h_vals = np.zeros_like(x)

    for n in range(N_max + 1):
        for k in range(2**n):
            h_vals += spike(x, n, k)
    
    return h_vals

In [ ]:
x_vals = np.linspace(0, 1, 1000)
h_vals = compute_h(x_vals, N_max=6)

plt.plot(x_vals, h_vals)
plt.title("Approximation of h(x) with N_max=6")
plt.xlabel("x")
plt.ylabel("h(x)")
plt.grid(True)
plt.show()

The function $h$ is a fractal function consisting of infinitely many non-overlapping spikes. Due to the choices of $h_n$, $w_n$, $a_{n,k}$, and $b_{n,k}$ used in the construction, the spikes become shorter and narrower as $n$ increases. As a result, it is straightforward to verify that $h$ is continuous.

The construction begins with a single spike of area $\frac{2}{3}$, followed by 2 spikes of area $\left(\frac{2}{3}\right)^2$, then 4 spikes of area $\left(\frac{2}{3}\right)^3$, and so on. A simple calculation shows that the total area under the graph of $h$ over $[0,1]$ is exactly 1.

This reasoning extends further. Let $C$ denote the Cantor middle-thirds set. Every $c \in C$ can be expressed as a base 3 expansion using only $0$s and $2$s, or equivalently as a linear combination of powers of $\frac{2}{3}$. It easily follows that, given any such $c$, we can construct a point $x \in [0,1]$ such that the cumulative area under $h$ from $0$ to $x$ is exactly $c$. This defines a bijection between $C$ and the set of zeros of $h$, where $h(x) = 0$ if and only if the cumulative area up to $x$ equals $c$.


Motivated by this, we now define $H:[0,1]\rightarrow [0,1]$ as follows:

\begin{equation}\nonumber
    H(x):=\int_{t=0}^x h(x)dt,
\end{equation}
and extend this definition to $[-1,1]$ by taking $H(x):=H(-x)$ for $x<0$. Note that, as $H'(0)=h(0)=0$, $H$ is continuously differentiable across the whole of $[-1,1]$.

In [ ]:
def compute_H(x_vals, normalise=True):
    
    # Separate into negative and non-negative parts
    x_neg = x_vals[x_vals < 0]
    x_pos = x_vals[x_vals >= 0]

    # Prepare result array
    H_vals = np.empty_like(x_vals, dtype=float)

    # Compute H for negative values
    if len(x_neg) > 0:

        # Flip sign and reverse order
        x_neg_flipped = -x_neg[::-1]  

        # Compute h values for negative
        h_neg = compute_h(x_neg_flipped, N_max=6)

        # Compute integral for negative
        H_neg = compute_integral(x_neg_flipped, h_neg)
        
        # Reverse to match original x_neg order
        H_neg = H_neg[::-1]  

        # Insert back into array
        H_vals[:len(H_neg)] = H_neg

    # Compute H for positive values
    if len(x_pos) > 0:
        
        # Compute h values for positive
        h_pos = compute_h(x_pos, N_max=6)
        
        # Compute integral for positive
        H_pos = compute_integral(x_pos, h_pos)

        # Insert back into array
        H_vals[-len(H_pos):] = H_pos

    # Normalise entire result if requested
    if normalise:
        H_vals = H_vals / H_vals[-1]

    # Return results
    return H_vals


In [ ]:
x_vals = np.linspace(0, 1, 1000)
H_vals = compute_H(x_vals)

plt.plot(x_vals, H_vals)
plt.title("Approximation of H(x)")
plt.xlabel("x")
plt.ylabel("h(x)")
plt.grid(True)
plt.show()

In [ ]:
x_vals = np.linspace(-1, 1, 1000)
H_vals = compute_H(x_vals)

plt.plot(x_vals, H_vals)
plt.title("Approximation of H(x)")
plt.xlabel("x")
plt.ylabel("h(x)")
plt.grid(True)
plt.show()

By construction, $H$ is increasing and has critical points at every level in $C$. 

Next, we define $H^{*}:[-1,1]^2\rightarrow [-1,1]$ by $H(x,y):=\frac{1}{2}(H(x)+H(y))$. 

In [ ]:
# Function to compute H
def compute_H_star(x_vals,y_vals=None):

    # Assume we are using the same axes for x and y if not given different
    if y_vals is None:
        y_vals = x_vals
        
    # Compute H_star values for one slice
    H_star_vals_slice = compute_H(x_vals)
    
    # Create 2D grid (X, Y) and compute H(|x|) + H(|y|)
    X, Y = np.meshgrid(x_vals, x_vals, indexing='xy')
    H_star_vals = np.zeros_like(X)
    
    # Compute number of points
    num_points = len(x_vals)
    
    # Compute H_star for whole surface
    for i in range(num_points):
        for j in range(num_points):
            H_star_vals[i, j] = (H_star_vals_slice[i] + H_star_vals_slice[j])/2

    return(H_star_vals)

# Compute H_star_vals
H_star_vals = compute_H_star(x_vals)

In [ ]:
# Example usage
plot_surface_3d(H_star_vals, x_vals, downsample=2, title="3D Surface With Critical Values [0,1]", z_title='H*(x,y) = H(|x|) + H(|y|)')

Through exploiting a well known function of the Cantor set; $C+C=[0,2]$, it can be seen that $H$ has a critical value at every level $z\in[0,1]$.

Before moving onto step 2, we note that it is now possible to construct a function $f:[0,1]^3\rightarrow \mathbb{R}$ with the below properties:

 - $f(0,0,0)=0$.
 - The only critical point of $f$ occurs at the origin.
 - for every $\epsilon > 0$, the zero level set $f^{-1}(0)$ intersects the plane $z=\epsilon$ tangentially.

An example of such a function is given by:

\begin{equation}\nonumber
f(x,y,z):=(z-H^{*}(x,y))(x^2+y^2+z^2).
\end{equation}

However, to construct an example in which the plane $z=\epsilon$ in the final condition is replaced by $B_\epsilon$ is difficult, and needs a little more care.

### Step 2: Constructing $\tilde{f}$

To construct a surface which is tangent to every ball of radius $\epsilon$, rather than every plane of height $\epsilon$, we now shift to a spherical coordinate framework.

Previously, we considered the surface defined by $z = H(x, y)$ in Cartesian coordinates, where $(x, y) \in [-1, 1]^2$. We now consider instead $r = H^*(\theta, \psi)$, where $(\theta, \psi) \in [\pi/4, 3\pi/4] \times [\pi/2, 3\pi/2]$. This change implicitly rescales the input domain of $H^*$, stretching the original Cartesian domain by factors of $\pi/2$ and $\pi$ and translating along the $x$- and $y$-axes, respectively.

For the single one dimensional slice of $H^*$ given by $h(x)=H^*(x,0)$, this transformation reduces to a polar transformation. The transformed function is depicted below, alongside the cartesian counterpart. Note how flat regions (where the Cartesian gradient vanishes) correspond to directions in which the radial derivative vanishes in the polar setting.

In [ ]:
# Cartesian plot
x_vals = np.linspace(-1, 1, 1000)
h_vals = compute_H(x_vals)

# Polar data
theta_vals = (x_vals/2+1)*np.pi
r_vals = compute_H(x_vals) 

# Set up the figure
fig = plt.figure(figsize=(12, 5))

# Cartesian subplot
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(x_vals, h_vals)
ax1.set_title("Cartesian: y = h(x)")
ax1.set_xlabel("x")
ax1.set_ylabel("h(x)")
ax1.grid(True)

# Polar subplot
ax2 = fig.add_subplot(1, 2, 2, projection='polar')
ax2.plot(theta_vals, r_vals)
ax2.set_title("Polar: r = h(θ)")

plt.tight_layout()
plt.show()

In the following we shall consider only what happens to this slice for ease of visualisation. Before this however, gor completeness, the full 3D surface obtained under the spherical transformation is visualised below.

In [ ]:
# Compute theta and psi
theta_vals = (x_vals/2 + 1) * (np.pi/4)     # [π/4, 3π/4]
psi_vals = (x_vals/2 + 1) * np.pi/4            # [π/2, 3π/2]

# Construct grid of (theta,psi) values
Theta, Psi = np.meshgrid(theta_vals, psi_vals, indexing='ij')  # shape (N, N)

# Compute R values on this grid
R = compute_H_star(x_vals, x_vals)  

# Construct coordinates for visualisation
X = R * np.sin(Theta) * np.cos(Psi)
Y = R * np.sin(Theta) * np.sin(Psi)
Z = R * np.cos(Theta)

# Plot surface, sphere added to demonstrate tangency
plot_surface_3d(Z, X, Y, downsample=2,sphere_radius=2/3)

Our goal is to construct a $C^1$ function $\tilde{f}$ that has the desired surface as a level set, and which has no critical points except at the origin.

A natural question is whether this can be achieved by simply composing our previously defined function $f$ with the inverse of the spherical coordinate transformation — i.e., applying $f$ in spherical coordinates rather than Cartesian. This approach can indeed be used to produce a function with the correct level set.

However, a problem arises: under this transformation, all points of the form $(x, y, 0)$ in Cartesian space are mapped to $(\theta, \phi, r = 0)$—that is, the origin. If the original Cartesian function $f$ satisfies $f(x, y, 0) \neq f(x', y', 0)$ for some $(x, y) \neq (x', y')$, then its composition with the spherical coordinate transformation will be discontinuous at the origin. In particular, it cannot be $C^1$ there.

We must instead take some care here. To construct $\tilde{f}$, we shall return to the the single slice $h$ and focus on the interval $[0,1]$. We shall now construct a new function $G:[0,1]\rightarrow[0,1]$, with the following desirable properties:

 - $G(x)\leq H(x)$ for all $x\in[0,1]$ with equality holding only when $x=0$.
 - $G'(x)\geq 0$ for all $x\in[0,1]$ with equality holding only when $x \in \{0,1\}$.
 - $\left(\frac{G}{H}\right)'(0)=0$

In [ ]:
def construct_new_vals_thickened(H_vals, k_vals, x_grid, y_grid, epsilon=1.0):
    """
    Constructs new_vals with correct epsilon-thickening logic:
      - new_vals = H_vals inside [-1, 1]^2
      - new_vals = T in thickened shell (distance <= epsilon, not box)
      - new_vals = k_vals outside the thickened region

    Parameters:
    - H_vals: np.ndarray, values of H(x, y)
    - k_vals: np.ndarray, values of k(x, y)
    - x_grid, y_grid: coordinate grids matching H_vals shape
    - epsilon: thickness parameter for blending

    Returns:
    - new_vals: np.ndarray, constructed values
    """

    new_vals = np.zeros_like(H_vals)

    # Step 1: Compute distance from each point to [-1,1]^2 box
    dx = np.maximum(np.abs(x_grid) - 1, 0)
    dy = np.maximum(np.abs(y_grid) - 1, 0)
    dist = np.sqrt(dx**2 + dy**2)

    # Step 2: Define regions
    in_core = (np.abs(x_grid) <= 1) & (np.abs(y_grid) <= 1)
    in_thickened = (dist <= epsilon) & (~in_core)
    outside = dist > epsilon

    # Step 3: Compute blending weights in shell region
    d = dist
    denom = d**2 + (1 - d)**2
    denom[denom == 0] = 1  # Avoid division by zero
    w = d**2 / denom

    # Step 4: Assign values
    new_vals[in_core] = H_vals[in_core]
    new_vals[in_thickened] = w[in_thickened] * k_vals[in_thickened] + (1 - w[in_thickened]) * H_vals[in_thickened]
    new_vals[outside] = k_vals[outside]

    return new_vals


In [ ]:
# x values
x_vals = np.linspace(-10,10,1000)
y_vals = np.linspace(-5,5,1000)

# Compute H_star_vals
H_star_vals = compute_H_star(x_vals,y_vals)

# Constant values
c_vals = np.ones(H_star_vals.shape)

# Example usage
# plot_surface_3d(H_star_vals, x_vals, y_vals, downsample=2, title="3D Surface With Critical Values [0,1]", z_title='H*(x,y) = H(|x|) + H(|y|)')

In [ ]:
# Generate 2D grids from x_vals
X, Y = np.meshgrid(x_vals, y_vals)

new_vals = construct_new_vals_thickened(H_star_vals, c_vals, X, Y, epsilon=1.0)


# Example usage
plot_surface_3d(new_vals, x_vals, downsample=2, title="3D Surface With Critical Values [0,1]", z_title='H*(x,y) = H(|x|) + H(|y|)')

In [ ]:
print(H_star_vals.shape, c_vals.shape, new_vals.shape, x_vals.shape)

In [ ]:
# Compute theta and psi
theta_vals = np.linspace(0,1,len(x_vals))*np.pi#(x_vals/2 + 1) * (np.pi/4)     # [π/4, 3π/4]
psi_vals = np.linspace(0,2,len(x_vals))*np.pi           # [π/2, 3π/2]

# Construct grid of (theta,psi) values
Theta, Psi = np.meshgrid(theta_vals, psi_vals, indexing='ij')  # shape (N, N)

# Compute R values on this grid
R = new_vals 

# Construct coordinates for visualisation
X = R * np.sin(Theta) * np.cos(Psi)
Y = R * np.sin(Theta) * np.sin(Psi)
Z = R * np.cos(Theta)

# Plot surface, sphere added to demonstrate tangency
plot_surface_3d(Z, X, Y, downsample=2,sphere_radius=None)

In [ ]:
X_masked = np.array(X)
Y_masked = np.array(Y)
Z_masked = np.array(Z)

X_masked[R>0.4]=0
Y_masked[R>0.4]=0
Z_masked[R>0.4]=0

# Plot surface, sphere added to demonstrate tangency
plot_surface_3d(Z_masked, X_masked, Y_masked, downsample=1,sphere_radius=None)